In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

sns.set()
sns.set_style('whitegrid')

In [ ]:
algorithms = [
    ('AceHash', ['Lambda', 'Alpha']),
    ('BBHash', ['Gamma']),
    ('PTHash', ['C', 'Alpha']),
    ('VectorMap', []),
    ('std::unordered_map', []),
    ('absl::flat_hash_map', []),
    ('folly::F14FastMap', []),
    ('DuckDB', [])
]

def read_csv(file_name):
    df = pd.read_csv(file_name)
    
    result = []
    for family, parameters in algorithms:
        df_algorithm = df[df['Algorithm'].str.split(';').str[0] == family].copy()
        for i, parameter in enumerate(parameters):
            df_algorithm[parameter] = df['Parameters'].str.split(';').str[i].astype(float)
        
        result.append(df_algorithm)
            
    return pd.concat(result)

# Baselines

In [ ]:
figsize=((4.8, 2.8))

df = pd.read_csv('csv/bench_baseline.csv')
df['Family'] = df['Algorithm'].str.split('(').str[0]
df['Construction throughput'] = 10e6 / df['Build time (s)'] / 1e6
df['Evaluation throughput'] = 10e6 / df['Probe time (s)'] / 1e6
   
fig, ax = plt.subplots(figsize=figsize)
sns.scatterplot(
    ax=ax,
    data=df,
    x='Construction throughput',
    y='Evaluation throughput',
    hue='Family'
)
ax.set_xlabel('Construction throughput\n(million keys per second)')
ax.set_ylabel('Evaluation throughput\n(million keys per second)')
ax.set_xlim((0, 13.5))
ax.set_ylim((0, 35))
ax.legend(title='PHF scheme', loc=(1.05, 0.17))
fig.subplots_adjust(left=0.19, right=0.7, bottom=0.25, top=0.97)
fig.savefig('pdf/results-baselines-probe-vs-build.pdf')

## Parameter exploration

In [ ]:
figsize=((3.3, 2.6))
boundaries = {'left': 0.26, 'right': 0.97, 'bottom': 0.24, 'top': 0.9}
options = {'linewidth': 2, 'marker': 'o'}
# y_axis_label_coords = (-0.21, 0.5)
x_ticks = [1.5, 2, 2.5, 3, 3.5, 4, 4.5]

df = read_csv('csv/bench_function.csv')
df = df[df['Trial'] >= 2]

df = df[
    (df['Experiment'] == 1)
    & (df['Algorithm'] == 'AceHash;1;1;1;0')
    & df['Alpha'].isin([0.7, 0.8, 0.9, 0.95, 1])
]

df['Number of bits per key'] = df['Number of bits'] / df['Number of keys']
df['Construction throughput'] = df['Number of keys'] / df['Construction time (s)'] / 1e6
df['Serial evaluation throughput'] = 100e6 / df['Serial evaluation time (s)'] / 1e6
df['Parallel evaluation throughput'] = 100e6 / df['Parallel evaluation time (s)'] / 1e6

fig, ax = plt.subplots(figsize=figsize)
sns.lineplot(
    ax=ax,
    data=df,
    x='Lambda',
    y='Construction throughput',
    hue='Alpha',
    **options
)
ax.set_xlabel('$\\lambda$')
ax.set_xticks(x_ticks)
ax.set_ylabel('Construction throughput\n(million keys per second)')
ax.legend([], [], frameon=False)
fig.subplots_adjust(**boundaries)
fig.savefig('pdf/results-params-construction-vs-lambda.pdf')

fig, ax = plt.subplots(figsize=figsize)
sns.lineplot(
    ax=ax,
    data=df,
    x='Lambda',
    y='Serial evaluation throughput',
    hue='Alpha',
    **options
)
ax.set_xlabel('$\\lambda$')
ax.set_xticks(x_ticks)
ax.set_ylabel('Evaluation throughput\n(million keys per second)')
ax.legend([], [], frameon=False)
fig.subplots_adjust(**boundaries)
fig.savefig('pdf/results-params-evaluation-vs-lambda.pdf')

fig, ax = plt.subplots(figsize=figsize)
sns.lineplot(
    ax=ax,
    data=df,
    x='Lambda',
    y='Number of bits per key',
    hue='Alpha',
    **options
)
ax.set_xlabel('$\\lambda$')
ax.set_ylabel('Size (bits per key)')
ax.set_xticks(x_ticks)
ax.legend([], [], frameon=False)
fig.subplots_adjust(**boundaries)
fig.savefig('pdf/results-params-size-vs-lambda.pdf')

fig = plt.figure(figsize=(4.8, 0.55))
fig.legend(*ax.get_legend_handles_labels(), 'center', ncol=5, title='$\\alpha$')
fig.savefig('pdf/results-params-legend.pdf', transparent=True)

## Optimizations

In [ ]:
figsize=((4.8, 2.4))
boundaries = {'left': 0.32, 'right': 0.98, 'bottom': 0.3, 'top': 0.96}
options = {'y': 'Algorithm', 'hue': 'Family', 'orient': 'h', 'legend': False}

df = read_csv('csv/bench_function.csv')

df = df[df['Trial'] >= 2]
df['Family'] = df['Algorithm'].str.split(';').str[0]
df['Algorithm'] = df['Algorithm'].replace({
    'AceHash;0;0;0;0': 'AceHash-O1',
    'AceHash;1;0;0;0': 'AceHash-O2',
    'AceHash;1;1;0;0': 'AceHash-O3',
    'AceHash;1;1;0;1': 'AceHash-O4',
    'AceHash;1;1;1;1': 'AceHash-O5',
    'BBHash;0': 'BBHash-Serial',
    'BBHash;1': 'BBHash-Parallel',
    'PTHash;dictionary-dictionary;1': 'PTHash'
})

df = df[
    (df['Algorithm'] != 'AceHash;1;1;1;0')
    & (df['Experiment'] == 2)
    & (df['Number of keys'] == 10e6)
    & df['Parameters'].isin(['2.5;1', '2.000000', '8;0.98'])
]

df['Number of bits per key'] = df['Number of bits'] / df['Number of keys']
df['Construction throughput'] = df['Number of keys'] / df['Construction time (s)'] / 1e6
df['Serial evaluation throughput'] = 100e6 / df['Serial evaluation time (s)'] / 1e6
df['Parallel evaluation throughput'] = 100e6 / df['Parallel evaluation time (s)'] / 1e6

df_evaluation = df[df['Algorithm'] != 'BBHash-Parallel'].copy()
df_evaluation['Algorithm'] = df_evaluation['Algorithm'].replace({'BBHash-Serial': 'BBHash'})

fig, ax = plt.subplots(figsize=figsize)
sns.barplot(
    ax=ax,
    data=df,
    x='Construction throughput',
    **options
)
ax.set_xlabel('Construction throughput\n(millions of keys per second)')
fig.subplots_adjust(**boundaries)
fig.savefig('pdf/results-micro-construction-vs-algo.pdf')

fig, ax = plt.subplots(figsize=figsize)
sns.barplot(
    ax=ax,
    data=df_evaluation,
    x='Serial evaluation throughput',
    **options
)
ax.set_xlabel('Serial evaluation throughput\n(millions of keys per second)')
fig.subplots_adjust(**boundaries)
fig.savefig('pdf/results-micro-serial-evaluation-vs-algo.pdf')
    
fig, ax = plt.subplots(figsize=figsize)
sns.barplot(
    ax=ax,
    data=df_evaluation,
    x='Parallel evaluation throughput',
    **options
)
ax.set_xlabel('Parallel evaluation throughput\n(millions of keys per second)')
fig.subplots_adjust(**boundaries)
fig.savefig('pdf/results-micro-parallel-evaluation-vs-algo.pdf')

df = df.groupby('Algorithm').agg({
    'Construction throughput': 'mean',
    'Serial evaluation throughput': 'mean',
    'Parallel evaluation throughput': 'mean'
}).transpose()

print(df)

for level in range(1, 6):
    df[f'AceHash-O{level} vs BBHash-Serial'] = df[f'AceHash-O{level}'] / df['BBHash-Serial']
    df[f'AceHash-O{level} vs BBHash-Parallel'] = df[f'AceHash-O{level}'] / df['BBHash-Parallel']
    df[f'AceHash-O{level} vs PTHash'] = df[f'AceHash-O{level}'] / df['PTHash']

print(df['AceHash-O5 vs BBHash-Parallel'])
print(df['AceHash-O5 vs PTHash'])

## Retrieval

In [ ]:
figsize=((3.3, 2.6))
boundaries = {'left': 0.28, 'right': 0.97, 'bottom': 0.24, 'top': 0.86}
options = {
    'x': 'Number of keys',
    'hue': 'Algorithm',
    'style': 'Algorithm',
    'linewidth': 2,
    'markers': True
}
y_axis_label_coords = (-0.19, 0.5)

df = read_csv('csv/bench_join.csv')

df = df[df['Algorithm'] != 'DuckDB']

df['Algorithm'] = df['Algorithm'].replace({
    'AceHash;1;1;1;0': 'AceHash',
    'BBHash;0': 'BBHash',
    'PTHash;dictionary-dictionary;1': 'PTHash',
    'std::unordered_map': 'STL',
    'absl::flat_hash_map': 'Abseil',
    'folly::F14FastMap': 'Folly'
})

df['Number of bits per key'] = df['Number of bits'] / df['Number of keys']
df['Construction throughput'] = df['Number of keys'] / df['Construction time (s)'] / 1e6
df['Serial evaluation throughput'] = 100e6 / df['Serial evaluation time (s)'] / 1e6
df['Parallel evaluation throughput'] = 100e6 / df['Parallel evaluation time (s)'] / 1e6

def plot_retrieval(value_type):
    fig, ax = plt.subplots(figsize=figsize)
    sns.lineplot(
        ax=ax,
        data=df[df['Value type'] == value_type],
        y='Construction time (s)',
        **options
    )
    ax.set_xscale('log')
    ax.set_ylabel('Construction time\n(seconds)')
    ax.legend([], [], frameon=False)
    fig.subplots_adjust(**boundaries)
    fig.savefig(f'pdf/results-retrieval-{value_type}-construction-vs-number.pdf')

    fig, ax = plt.subplots(figsize=figsize)
    sns.lineplot(
        ax=ax,
        data=df[df['Value type'] == value_type],
        y='Serial evaluation throughput',
        **options
    )
    ax.set_xscale('log')
    ax.set_ylabel('Serial retrieval throughput\n(million keys per second)')
    ax.legend([], [], frameon=False)
    fig.subplots_adjust(**boundaries)
    fig.savefig(f'pdf/results-retrieval-{value_type}-serial-evaluation-vs-number.pdf')

    fig, ax = plt.subplots(figsize=figsize)
    sns.lineplot(
        ax=ax,
        data=df[df['Value type'] == value_type],
        y='Parallel evaluation throughput',
        **options
    )
    ax.set_xscale('log')
    ax.set_ylabel('Parallel retrieval throughput\n(million keys per second)')
    ax.legend([], [], frameon=False)
    fig.subplots_adjust(**boundaries)
    fig.savefig(f'pdf/results-retrieval-{value_type}-parallel-evaluation-vs-number.pdf')

    fig = plt.figure(figsize=(7.1, 0.55))
    fig.legend(*ax.get_legend_handles_labels(), 'center', ncol=6, title='Method')
    fig.savefig('pdf/results-retrieval-legend.pdf', transparent=True)

plot_retrieval('m')
plot_retrieval('h')

In [ ]:
df = read_csv('csv/bench_join.csv')

df = df[df['Algorithm'] != 'DuckDB']

df['Algorithm'] = df['Algorithm'].replace({
    'AceHash;1;1;1;0': 'AceHash',
    'BBHash;0': 'BBHash',
    'PTHash;dictionary-dictionary;1': 'PTHash',
    'std::unordered_map': 'STL',
    'absl::flat_hash_map': 'Abseil',
    'folly::F14FastMap': 'Folly'
})

df = df[df['Value type'] == 'm'].copy()

df = df.groupby(['Number of keys', 'Algorithm']).agg({
    'Construction time (s)': 'mean',
    'Serial evaluation time (s)': 'mean',
    'Parallel evaluation time (s)': 'mean',
}).unstack()


print(df['Construction time (s)']['Abseil'] / df['Construction time (s)']['AceHash'])
print(df['Serial evaluation time (s)']['Abseil'] / df['Serial evaluation time (s)']['AceHash'])
print(df['Parallel evaluation time (s)']['Abseil'] / df['Parallel evaluation time (s)']['AceHash'])

## Join

In [ ]:
figsize=((3.3, 3.2))
boundaries = {'left': 0.17, 'right': 0.97, 'bottom': 0.24, 'top': 0.86}
options = {
    'x': 'Number of keys',
    'hue': 'Algorithm',
    'style': 'Algorithm',
    'linewidth': 2,
    'markers': True
}

df = read_csv('csv/bench_join.csv')

df['Algorithm'] = df['Algorithm'].replace({
    'AceHash;1;1;1;0': 'AceHash',
    'PTHash;dictionary-dictionary;1': 'PTHash',
    'std::unordered_map': 'STL',
    'absl::flat_hash_map': 'Abseil',
    'folly::F14FastMap': 'Folly'
})

df = df[
    (df['Algorithm'] != 'STL')
    & (df['Algorithm'] != 'Folly')
    & (df['Algorithm'] != 'BBHash;0')
]

df['Query time (s)'] = df['Construction time (s)'] + df['Serial evaluation time (s)']

def plot_join(value_type):
    speedups = df[df['Value type'] == value_type].groupby(['Number of keys', 'Algorithm']).agg({'Query time (s)': 'mean'}).unstack()
    speedups = speedups['Query time (s)'][['PTHash', 'Abseil', 'DuckDB']].min(axis=1) / speedups['Query time (s)']['AceHash']
    print(speedups)
    
    fig, (ax1, ax2) = plt.subplots(2, figsize=figsize, sharex=True, gridspec_kw={'height_ratios': [5, 2]})
    sns.lineplot(
        data=df[df['Value type'] == value_type],
        ax=ax1,
        y='Query time (s)',
        **options
    )
    ax1.set_xscale('log')
    ax1.set_yscale('log')
    ax1.set_ylim((0.45 if value_type == 'h' else 0.55, 100))
    ax1.set_ylabel('Query time\n(seconds)')
    ax1.legend([], [], frameon=False)

    sns.lineplot(
        data=speedups,
        ax=ax2,
        linewidth=2,
        marker='o'
    )
    ax2.set_xscale('log')
    ax2.set_xlabel('Number of build tuples')
    ax2.set_ylim((0.6, 3.0) if value_type == 'h' else (0.5, 2.3))
    ax2.set_ylabel('Speedup\nover best')

    fig.align_labels()
    fig.subplots_adjust(
        left=0.25,
        right=0.98,
        bottom=0.18,
        top=0.95
    )
    fig.savefig(f'pdf/results-query-join-{value_type}-time-vs-number.pdf')
    
plot_join('h')
plot_join('m')

## Aggregate

In [ ]:
figsize=((3.3, 3.2))
options = {
    'x': 'Key cardinality',
    'hue': 'Algorithm',
    'style': 'Algorithm',
    'linewidth': 2,
    'markers': True
}

df = read_csv('csv/bench_aggregate.csv')

df['Algorithm'] = df['Algorithm'].replace({
    'AceHash;1;1;1;0': 'AceHash',
    'PTHash;dictionary-dictionary;1': 'PTHash',
    'std::unordered_map': 'STL',
    'absl::flat_hash_map': 'Abseil',
    'folly::F14FastMap': 'Folly'
})

df = df[
    (df['Algorithm'] != 'STL')
    & (df['Algorithm'] != 'Folly')
    & (df['Algorithm'] != 'BBHash;0')
    & (df['Value type'] == 'm')
]

df['Query time (s)'] = df['Build time (s)'] + df['Serial aggregate time (s)']

speedups = df.groupby(['Key cardinality', 'Algorithm']).agg({'Query time (s)': 'mean'}).unstack()
speedups = speedups['Query time (s)'][['PTHash', 'Abseil', 'DuckDB']].min(axis=1) / speedups['Query time (s)']['AceHash']
print(speedups)

fig, (ax1, ax2) = plt.subplots(2, figsize=figsize, sharex=True, gridspec_kw={'height_ratios': [5, 2]})

sns.lineplot(
    data=df,
    ax=ax1,
    y='Query time (s)',
    **options
)
ax1.set_xscale('log')
ax1.set_yscale('log')
ax1.set_ylim((0.5, 100))
ax1.set_ylabel('Query time\n(seconds)')
ax1.legend([], [], frameon=False)

sns.lineplot(
    data=speedups,
    ax=ax2,
    linewidth=2,
    marker='o'
)
ax2.set_xscale('log')
ax2.set_xlabel('Number of groups')
ax2.set_ylim((0.3, 2.8))
ax2.set_ylabel('Speedup\nover best')

fig.align_labels()
fig.subplots_adjust(
    left=0.25,
    right=0.98,
    bottom=0.18,
    top=0.95
)
fig.savefig('pdf/results-query-aggregate-time-vs-number.pdf')

fig = plt.figure(figsize=(5.0, 0.55))
fig.legend(*ax1.get_legend_handles_labels(), 'center', ncol=6, title='Method')
fig.savefig('pdf/results-query-legend.pdf', transparent=True)

## Intro

In [ ]:
figsize=(4.8, 2.0)

df = pd.read_csv('csv/bench_intro.csv')
df = df[df['Alpha'] >= 0.2]
df['Throughput'] = 10e6 / df['Time (s)'] / 1e6

fig, ax = plt.subplots(figsize=figsize)
sns.lineplot(
    data=df,
    x='Alpha',
    y='Throughput',
    hue='Algorithm',
    style='Algorithm',
    linewidth=2
)
ax.set_xlabel('Load factor')
ax.set_ylabel('Throughput (million\nlookups per second)')
ax.legend(title='Algorithm', loc=(1.03, 0.22))
fig.subplots_adjust(left=0.165, right=0.68, bottom=0.24, top=0.94)
fig.savefig('pdf/results-intro.pdf')

print(df[df['Algorithm'] == 'Conventional']['Throughput'].max())
print(df[df['Algorithm'] == 'AceHash']['Throughput'].max())